In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import  Select
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
# import numpy as np
import time

position = {'QB': 2, 'RB': 3, 'WR': 4, 'TE': 5, 'D/ST': 7, 'K': 8}
week = {"NFL Week 1": 6, "NFL Week 2": 5, "NFL Week 3": 4, "NFL Week 4": 3}

driver = webdriver.Chrome()
driver.get("https://fantasy.espn.com/football/leaders")

In [2]:

def extract_table_to_dataframe(header_xpath: str, tBody_xpath: str) -> pd.DataFrame:
    # Extract the column names from the header row
    header_row = driver.find_element(By.XPATH, header_xpath)
    cols = header_row.find_elements(By.TAG_NAME, 'th')
    column_names = [col.text for col in cols]
    df = pd.DataFrame(columns=column_names)
    tbody = driver.find_element(By.XPATH, tBody_xpath)
    rows = tbody.find_elements(By.TAG_NAME, 'tr')
    for row in rows:
        temp_list = [col.text for col in row.find_elements(By.TAG_NAME, 'td')]
        df.loc[len(df)] = temp_list
    return df

In [3]:

final_df = pd.DataFrame()

time.sleep(4)
for week, week_index in week.items():
    dropdown = driver.find_element(By.XPATH, f'//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[2]/div[1]/div/div[2]/div[5]/div/div[2]/select/option[{week_index}]')
    print(f'Week: {week}')
    dropdown.click()
    time.sleep(3)

    # Loop through positions
    for position, position_index in position.items():
        print(f'Position: {position}')
        label = driver.find_element(By.XPATH, f'//*[@id="filterSlotIds"]/label[{position_index}]')
        label.click()
        
        while True: 
            i = 0
            try: 
                time.sleep(6)
                start = time.time()
                header_row = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/table[1]/thead/tr[2]'
                tbody = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/table[1]/tbody'
                df1=extract_table_to_dataframe(header_xpath=header_row, tBody_xpath=tbody)
                header_row = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/div/div[2]/table/thead/tr[2]'
                tbody = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/div/div[2]/table/tbody'
                df2=extract_table_to_dataframe(header_xpath=header_row, tBody_xpath=tbody)
                header_row = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/table[2]/thead/tr[2]'
                tbody = '//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/div/div/table[2]/tbody'
                df3=extract_table_to_dataframe(header_xpath=header_row, tBody_xpath=tbody)
                df_combined = pd.concat([df1, df2, df3], axis=1)
                df_combined['week'] = str(week)
                final_df = final_df.append(df_combined, ignore_index=True)

                button = driver.find_element(By.XPATH, f'//*[@id="fitt-analytics"]/div/div[5]/div[2]/div[3]/div/div/div/div/nav/button[2]')
                if button.is_enabled():
                    button.click()
                    print(f'Next page {time.time() - start}')
                else:
                    break
                pass
            except Exception as e:
                print(f'End of Position {time.time() - start}')
                break

Week: NFL Week 1
Position: QB
Going to next page for QB
Time taken for QB iter.: 13.943156003952026
Going to next page for QB
Time taken for QB iter.: 13.347488641738892
Position: RB
Going to next page for RB
Time taken for RB iter.: 13.499444961547852
Going to next page for RB
Time taken for RB iter.: 14.840734958648682
Going to next page for RB
Time taken for RB iter.: 17.467739820480347
Going to next page for RB
Time taken for RB iter.: 14.261007070541382
Going to next page for RB
Time taken for RB iter.: 13.174219846725464
Position: WR
Going to next page for WR
Time taken for WR iter.: 31.697915077209473
Going to next page for WR
Time taken for WR iter.: 18.134501934051514
Going to next page for WR
Time taken for WR iter.: 15.140248775482178
Going to next page for WR
Time taken for WR iter.: 14.698651790618896
Going to next page for WR
Time taken for WR iter.: 14.729641914367676
Going to next page for WR
Time taken for WR iter.: 14.547802209854126
Going to next page for WR
Time tak

ElementClickInterceptedException: Message: element click intercepted: Element <label class="control control--radio picker-option">...</label> is not clickable at point (326, 7). Other element would receive the click: <a class="AnchorLink Button--unstyled Nav__Primary__Menu__Link clr-white flex items-center ns7 ph3" tabindex="0" aria-label="nav.arialabel.homepage_%{title}" href="http://www.espn.com/nba/" id="nav-link-nav-menu-item-9748" aria-selected="false">...</a>
  (Session info: chrome=117.0.5938.132)


In [ ]:
final_df.head(200)

,PLAYER,TYPE,ACTION,OPP,STATUS,PROJ,C/A,YDS,TD,INT,...,TD,REC,YDS,TD,TAR,2PC,FUML,TD,FPTS,week
0,Josh Allen\nBuf\nQB,,,Mia,W 48-20,20.9,21/25,320,4,0,...,1,0,0,0,0,0,0,0,36.5,NFL Week 1
1,Anthony Richardson\nInd\nQB,,,LAR,L 23-29,20.2,11/25,200,2,0,...,1,0,0,0,0,0,1,0,29.6,NFL Week 1
2,Lamar Jackson\nBal\nQB,,,@Cle,W 28-3,17.9,15/19,186,2,0,...,2,0,0,0,0,0,1,0,28.14,NFL Week 1
3,Justin Fields\nChi\nQB,,,Den,L 28-31,20.0,28/35,335,4,1,...,0,0,0,0,0,0,1,0,27.9,NFL Week 1
4,Jalen Hurts\nPhi\nQB,,,Wsh,W 34-31,21.7,25/37,319,2,0,...,0,0,0,0,0,0,0,0,24.16,NFL Week 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Ben DiNucci\nFA\nQB,,,--,,--,--/--,--,--,--,...,--,--,--,--,--,--,--,--,--,NFL Week 1
196,Mike Glennon\nFA\nQB,,,--,,--,--/--,--,--,--,...,--,--,--,--,--,--,--,--,--,NFL Week 1
197,Matt Ryan\nFA\nQB,,,--,,--,--/--,--,--,--,...,--,--,--,--,--,--,--,--,--,NFL Week 1
198,Anthony Brown\nFA\nQB,,,--,,--,--/--,--,--,--,...,--,--,--,--,--,--,--,--,--,NFL Week 1
